# Preparação e Precessamento dos dados 


## Conferindo a integridade básica dos dados

Na etapa anterior, [Coleta e Importação dos Dados]("./03_coleta_e_importação_dos_dados.ipynb"), utilizamos o seguinte script para importar os dados e transformar o nome dos arquivos em um padrão adequado: 

In [1]:
import os
import re
import shutil
import pandas as pd

In [2]:
input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

MONTHS = {
    "1": "Janeiro",
    "2": "Fevereiro",
    "3": "Março",
    "4": "Abril",
    "5": "Maio",
    "6": "Junho",
    "7": "Julho",
    "8": "Agosto",
    "9": "Setembro",
    "10": "Outubro",
    "11": "Novembro",
    "12": "Dezembro"
}

pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")


def process_file(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith(".xls"):
            match = pattern.search(filename)
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                new_name = pattern.sub(f"{name_month}", filename).replace(".xls", ".csv")

                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
            for file in file_path_generator]

df = pd.concat(datasets, axis=0, ignore_index=True)

pd.set_option("display.max_columns", None)


/tmp/ipykernel_6351/1584778450.py:44: DtypeWarning: Columns (31,32,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_6351/1584778450.py:44: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_6351/1584778450.py:44: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")


Observe que, ao realizar a importação, recebemos um aviso `DtypeWarning`. Iremos começar verificar a consistência e integridade dos dados compreendendo a razão pela qual este aviso ocorre e checar como podemos processar os dados adequadamente para torná-los úteis para análise. 

In [ ]:
### Solucionando os problemas de tipo dos dados

A saída `DtypeWarning` do comando de importação nos avisa que o `pandas` não conseguiu identificar o tipo de algumas colunas do dataset e, por isso, teve que inferir um tipo. Em outras palavras, o `pandas` tenta determinar os tipos de cada coluna; uma operação que exige muita memória.  

Podemos silenciar esses avisos basicamente de duas formas: A primeira é utilizando o parâmetro `dtypes` e passando `object` como valor.`Object` [é a classe base de onde derivam toda as classes em Python](https://docs.python.org/pt-br/3/library/functions.html?highlight=object#object), ou seja, `object` pode conter qualquer objeto Python. Isso fará com que o `pandas` defina o tipo `object` para todos os dados. Esse tipo já é aplicado para tipos de dados mistos ou que o `pandas` não consegue inferir. [No entanto, essa medida deve ser evitado na medida do possível](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes) (para desempenho e interoperabilidade com outras bibliotecas e métodos).

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                dtypes=object) 
                for file in file_path_genarator]
```

A outra forma é usar o parâmetro `low-memory=False`, como sugere o próprio aviso: `DtypeWarning: Columns (...,...) have mixed types. Specify dtype option on import or set low_memory=False`.

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                low-memory=True) 
                for file in file_path_genarator]
```
 Porém, esse parâmetro, embora aparente funcionar, é considerado obsoleto e pode levar a erros silenciosos que não são documentados. [Existe uma discussão que se arrasta desde 2014 sobre se este parâmetro deve ou não ser melhor documentado ou retirado da biblioteca](https://github.com/pandas-dev/pandas/issues/5888). O fato é que sua utilização não é muito comum apesar do aviso indicar seu uso.

De toda forma, nenhuma das abordagens resolvem o problema de fato. O `pandas` continuará usando muito recurso para adivinhar os tipos. 

Portanto, a abordagem mais indicada nesses casos é fornecer os tipos manualmente. 

Antes de partir para esta tarefa, vamos checar o estado do DataFrame:


In [2]:
df.shape

(119158, 54)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ANO_BO                    119158 non-null  int64  
 1   NUM_BO                    119158 non-null  int64  
 2   NUMERO_BOLETIM            119158 non-null  object 
 3   BO_INICIADO               119158 non-null  object 
 4   BO_EMITIDO                119158 non-null  object 
 5   DATAOCORRENCIA            119158 non-null  object 
 6   HORAOCORRENCIA            110412 non-null  object 
 7   PERIDOOCORRENCIA          119158 non-null  object 
 8   DATACOMUNICACAO           119158 non-null  object 
 9   DATAELABORACAO            119158 non-null  object 
 10  BO_AUTORIA                119158 non-null  object 
 11  FLAGRANTE                 119158 non-null  object 
 12  NUMERO_BOLETIM_PRINCIPAL  37625 non-null   object 
 13  LOGRADOURO                111762 non-null  o


A saída do método `info()` nos fornece as seguintes informações:

**Column**: Esta coluna indica os nomes das colunas presentes no DataFrame.

**Non-Null Count**: Esta coluna mostra a contagem de valores não nulos em cada coluna. Em outras palavras, é o número de entradas que possuem valores válidos para aquela coluna. Por exemplo, na primeira coluna "ANO_BO", existem 119158 valores não nulos.

**Dtype**: Esta coluna informa o tipo de dados (data type) presente em cada coluna. Por exemplo, `object`, como vimos, indica que os registros podem qualquer objeto python, geralmente são strings ou valores mistos. Manter os dados assim pode causar inúmeros problemas já que strings e não-strings podem estar juntas em uma mesma série. Trabalhar com os tipos corretos no DataFrame é extremamente importante para que possamos garantir:

- Precisão nos dados
- Economia de memória
- Desempenho
- Manipulação dos dados mais eficaz
- Prevenção de erros
- Consistência. 

Podemos notar que, com exceção de alguns tipos `float64`, o pandas inferiu `object` em todo o conjunto de dados. Vamos olhar uma amostra aleatória dos registros para termos uma noção da tipagem que precisará ser adequada.

In [3]:
df.sample(10)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
76407,2023,383,383/2023,05/04/2023 19:42:38,05/04/2023 19:42:38,26/03/2023,00:30,DE MADRUGADA,04/04/2023,05/04/2023 19:42:38,Desconhecida,Não,379/2023 - 10360,RUA PADRE GABRIEL DE CAMPOS,0.0,ARTUR ALVIM,S.PAULO,SP,"-23,548604537","-46,486700281",Via pública,NaN,BO PARA INVESTIGAÇÃO,65º D.P. ARTUR ALVIM,65º D.P. ARTUR ALVIM,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Xiaomi
41868,2023,907326,907326/2023,11/06/2023 21:32:16,11/06/2023 21:32:21,11/06/2023,19:07,A NOITE,11/06/2023,11/06/2023 21:32:16,Desconhecida,Não,NaN,RUA ARGEU FULIOTO,1101.0,RIBEIRÂNIA,RIBEIRAO PRETO,SP,"-21,199296424","-47,779578773",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,04º D.P. RIBEIRAO PRETO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
60843,2023,230623,230623/2023,13/02/2023 09:43:55,13/02/2023 09:44:03,11/02/2023,09:30,PELA MANHÃ,11/02/2023,13/02/2023 09:43:55,Desconhecida,Não,NaN,RUA SAFIRA,73.0,LIBERDADE,S.PAULO,SP,"-23,5720638117272","-46,631815976909",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,06º D.P. CAMBUCI,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
21135,2023,737367,737367/2023,12/05/2023 10:43:02,12/05/2023 10:43:07,11/05/2023,20:30,A NOITE,11/05/2023,12/05/2023 10:43:02,Desconhecida,Não,NaN,Avenida Doutor Arnaldo,715.0,Pinheiros,S.PAULO,SP,"-23,5545234","-46,6735878",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,14º D.P. PINHEIROS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
77753,2023,100770,100770/2023,07/04/2023 20:04:13,07/04/2023 20:04:13,06/04/2023,19:50,A NOITE,06/04/2023,07/04/2023 20:04:13,Desconhecida,Não,NaN,RUA AVELINO BARREIROS,109.0,SACOMA,S.PAULO,SP,"-23,6254615334999","-46,5900009227143",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA 1,95º D.P. HELIÓPOLIS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - INTERIOR ESTABELECIMENTO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Outros
44005,2023,3258,3258/2023,14/06/2023 11:03:07,14/06/2023 11:03:07,12/06/2023,16:42,A TARDE,12/06/2023,14/06/2023 11:03:07,Desconhecida,Não,3246/2023 - 70346,NaN,0.0,JARDIM BELA VISTA,AMERICANA,SP,NaN,NaN,Residência,NaN,APRECIAÇÃO DO DELEGADO TITULAR,03º D.P. AMERICANA,03º D.P. AMERICANA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - RESIDENCIA,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QXS0J81,MG,BELO HORIZONTE,Preta,CHEVROLET/CRUZE LT NB,2019.0,2020.0,AUTOMOVEL,1.0,Samsung
178,2023,2032,2032/2023,01/01/2023 07:27:31,01/01/2023 07:27:31,01/01/2023,05:00,DE MADRUGADA,01/01/2023,01/01/2023 07:27:31,Desconhecida,Não,NaN,Rua Frei Caneca,401.0,Consolação,S.PAULO,SP,"-23,5528743","-46,6513184",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 1,04º D.P. CONSOLAÇÃO,Título II - Patrimôn

Ao observar os registros do DataFrame, podemos notar o seguinte:

1. Existem inconsistências nos registros: Além das datas estarem em formato inadequado para a maioria dos softwares e fora do padrão para dados,  alguns deles estão com caracteres maiúsculos e outros minúsculos. Será necessário padronizar os tipos de data e as strings e limpar os espaços extras. 
2. Os registros de `LATITUDE` e `LONGITUDE` possuem um espaço extra tanto à esquerda quanto à direita. Será necessário limpar estes espaços, substituir a "," por "." e converter o dados adequadamente para `float`.
3. A única coluna que se refere a valores do tipo `int` e que pode ser utilizada para cálculo é "QUANT_CELULAR", que se refere a quantidade de dispositivos roubados da vítima. Esta variável precisa ser convertida para o tipo adequado que suporte valores ausentes.
4. Existem muitos valores ausentes. Será necessário pensar em uma abordagem para lidar com eles.

Antes de partir para solução desses problemas, prefiro examinar as informações do dataset fornecidas pela Secretaria de Segurança. 


#### Conferindo o dicionário de dados 

Como uma tarefa complementar desta etapa, para facilitar a comparação entre o dicionário e o DataFrame, eu exportei a saída da propriedade `dtypes` e copiei a tabela do documento "METODOLOGIA" para uma planilha a fim de:

1. Checar se o DataFrame contém todas as variáveis presentes no dicionário de dados;
2. Identificar os tipos dos dados e atribuí-los adequadamente ao DataFrame.

A planilha se encontra no diretório `data` e, como poderá ser observado, fiz uma busca vertical (VLOOKUP ou PROCV) para cruzar as colunas que se referem às características do conjunto de dados. O código da exportação dos dtypes segue abaixo:

In [8]:
t = df.dtypes.reset_index()
t.columns = ["Variáveis", "Tipos"]
t.to_excel("./data/dtypes_out.xlsx", index=False)

Ao analisar o dicionário de dados, cheguei à conclusão de que ele oferece pouca ajuda. Os motivos são destacados a seguir:

1. O dicionário não informa os tipos dos dados;
2. As descrições das variáveis são vagas e não fornecem mais informações além do óbvio;
3. Os rótulos das variáveis especificadas no dicionário são diferentes dos rótulos do DataFrame, mas algumas se referem à mesma coisa. Por exemplo: `BOLETIM_EMITIDO`, uma das variáveis do DataFrame, não consta no dicionário, mas é equivalente à "DATAHORA_IMPRESSAO_BO" definida apenas como "Data-hora da elaboração da impressão" no dicionário.
4. O DataFrame possui 54 variáveis enquanto o dicionário indica 62. Dada a condição anterior, não consigo afirmar com certeza se o DataFrame possui variáveis não especificadas no dicionário de dados ou se as variáveis do dicionário se referem exclusivamente às do DataFrame.
5. O dicionário de dados informa que, várias linhas podem se referir ao mesmo boletim. Portanto, segundo o documento, para conclusões quanto as quantidades de ocorrências é necessária exclusão das duplicidades das seguintes variáveis:

- NOME_DELEGACIA
- ANO_BO
- NUM_BO

Isso será checado em detalhes posteriormente.


#### Corrigindo os tipos e limpando os dados

Para formatar corretamente os registros de Longitude e Latitude usaremos a seguinte função:

In [4]:
"""Limpa os registro LONGITUDE e LATITUDE """

def clean_coordinate_value(value):
    if isinstance(value, str):
        try:
            clean_value = value.strip().replace(",", ".")
            return float(clean_value)
        except ValueError:
            return None 
        
    return None

Para fornecer os tipos corretos dos registros, irei realizar uma nova importação utilizando os argumentos nomeados definidos no método `read_csv()`.

O método `read_csv()` possui diversos argumentos nomeados que podemos fornecer para cuidar da formatação dos registros. Estes argumentos precisam ser estruturas de dados específicas:
- `dtype`: Precisa ser um tipo, como `str`, ou um dicionário. Ele define os tipos de dados a serem aplicados a todo o conjunto de dados ou a colunas individuais
- `parse_dates`: `bool`, lista de Hashable, lista de listas ou dict de {Hashable list}, padrão `False`.  
- `date_format`: `str` ou `dict`  com nomes das colunas e um formato opcional
- `converters`: `dict` de {Hashable Callable}, optional. Aqui tem um peculiaridade: Se `converters` for definido, ele terá prioridade em vez do `dtype`.

Para passarmos essas estruturas para o método `read_csv()`, precisamos definilas:


In [26]:
DTYPES = {
    "ANO_BO": pd.StringDtype(),
    "ANO_FABRICACAO": pd.StringDtype(),
    "ANO_MODELO": pd.StringDtype(),
    "BO_AUTORIA": pd.StringDtype(),
    "CIDADE": pd.StringDtype(),
    "BAIRRO": pd.StringDtype(),
    "LOGRADOURO": pd.StringDtype(),
    "CIDADE_VEICULO": pd.StringDtype(),
    "CORCUTIS": None,
    "DELEGACIA_CIRCUNSCRICAO": pd.StringDtype(),
    "DELEGACIA_NOME": pd.StringDtype(),
    "DESCR_COR_VEICULO": pd.StringDtype(),
    "DESCR_MARCA_VEICULO": pd.StringDtype(),
    "DESCR_TIPO_VEICULO": pd.StringDtype(),
    "DESCRICAOLOCAL": pd.StringDtype(),
    "DESDOBRAMENTO": pd.StringDtype(),
    "ESPECIE": pd.StringDtype(),
    "ESTADOCIVIL": pd.StringDtype(),
    "EXAME": None,
    "FLAGRANTE": pd.StringDtype(),
    "GRAUINSTRUCAO": pd.StringDtype(),
    "IDADE": pd.Int64Dtype(),
    "MARCA_CELULAR": pd.StringDtype(),
    "NACIONALIDADE": pd.StringDtype(),
    "NATURALIDADE": pd.StringDtype(),
    "NATUREZAVINCULADA": pd.StringDtype(),
    "NUM_BO": pd.StringDtype(),
    "NUMERO": pd.StringDtype(),
    "NUMERO_BOLETIM": pd.StringDtype(),
    "NUMERO_BOLETIM_PRINCIPAL": pd.StringDtype(),
    "PARENTESCO": pd.StringDtype(),
    "PERIDOOCORRENCIA": pd.StringDtype(),
    "PLACA_VEICULO": pd.StringDtype(),
    "PROFISSAO": pd.StringDtype(),
    "QUANT_CELULAR": pd.Int64Dtype(),
    "RELACIONAMENTO": pd.StringDtype(),
    "RUBRICA": pd.StringDtype(),
    "SEXO": pd.StringDtype(),
    "SOLUCAO": pd.StringDtype(),
    "STATUS": pd.StringDtype(),
    "TIPOPESSOA": pd.StringDtype(),
    "TIPOVINCULO": pd.StringDtype(),
    "UF": pd.StringDtype(),
    "UF_VEICULO": pd.StringDtype(),
    "VITIMAFATAL": pd.StringDtype(),
}

DATE_TYPES = [
    "BO_EMITIDO",
    "BO_INICIADO",
    "DATACOMUNICACAO",
    "DATAELABORACAO",
    "DATANASCIMENTO",
    "DATAOCORRENCIA",
]

DATE_FORMAT = {
    "BO_EMITIDO": "%d/%m/%Y %H:%M:%S",
    "BO_INICIADO": "%d/%m/%Y %H:%M:%S",
    "DATACOMUNICACAO": "%d/%m/%Y",
    "DATAELABORACAO": "%d/%m/%Y %H:%M:%S",
    "DATANASCIMENTO": "%d/%m/%Y",
    "DATAOCORRENCIA": "%d/%m/%Y",
    "HORAOCORRENCIA": "%H:%M",
}

CONVERTERS = {
    "LATITUDE": clean_coordinate_value,
    "LONGITUDE": clean_coordinate_value,
}

TO_UPPER_LIST = [
    "ANO_BO",
    "ANO_FABRICACAO",
    "ANO_MODELO",
    "BO_AUTORIA",
    "CIDADE",
    "BAIRRO",
    "LOGRADOURO",
    "CIDADE_VEICULO",
    "DELEGACIA_CIRCUNSCRICAO",
    "DELEGACIA_NOME",
    "DESCR_COR_VEICULO",
    "DESCR_MARCA_VEICULO",
    "DESCR_TIPO_VEICULO",
    "DESCRICAOLOCAL",
    "DESDOBRAMENTO",
    "ESPECIE",
    "ESTADOCIVIL",
    "FLAGRANTE",
    "GRAUINSTRUCAO",
    "MARCA_CELULAR",
    "NACIONALIDADE",
    "NATURALIDADE",
    "NATUREZAVINCULADA",
    "NUM_BO",
    "NUMERO",
    "NUMERO_BOLETIM",
    "NUMERO_BOLETIM_PRINCIPAL",
    "PARENTESCO",
    "PERIDOOCORRENCIA",
    "PLACA_VEICULO",
    "PROFISSAO",
    "RELACIONAMENTO",
    "RUBRICA",
    "SEXO",
    "SOLUCAO",
    "STATUS",
    "TIPOPESSOA",
    "TIPOVINCULO",
    "UF",
    "UF_VEICULO",
    "VITIMAFATAL"
]


Poderíamos passar `str` para os campos do dict `DTYPES` referentes a strings, contudo, ainda teríamos registros avaliados como `object`. Portanto, a [maneira indicada de trabalharmos com registros de texto no `pandas` é utilizando a classe `StringDtype()`](https://pandas.pydata.org/docs/user_guide/text.html#string-methods). 

Outro ponto importante de destacar é a utilização da classe `Int64Dtype()` utilizada para a variável "QUANT_CELULAR". Usar `int`, neste caso, resultaria na exceção `ValueError: invalid literal for int() with base 10` que indica que o `pandas` não consegue converter uma string em um número. Essa confusão acontece, pois o `pandas` interpretou a variável como `object` para poder suportar valores nulos. Como a função embutida `int` é intolerante e levanta uma exceção quando esbarra com um valor que não consegue converter (como valores nulos), precisamo usar `Int64Dtype()` que é um tipo de numeral inteiro anulável. 

Por fim, a constante `TO_UPPER_LIST` será utilizada para transformar os caracteres minúsculos em maiúsculo sem que o tipo da `Serie` volte a ser `object`.

In [24]:
"""
Padroniza os nomes dos arquivos para "DadosBO_SP_{ano}_{mes}.xls
"""
def process_file(input_dir, output_dir):

    PATTERN = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")

    MONTHS = {
        "1": "Janeiro",
        "2": "Fevereiro",
        "3": "Março",
        "4": "Abril",
        "5": "Maio",
        "6": "Junho",
        "7": "Julho",
        "8": "Agosto",
        "9": "Setembro",
        "10": "Outubro",
        "11": "Novembro",
        "12": "Dezembro"
    }

    for filename in os.listdir(input_dir):
        # Verificar se o arquivo é um arquivo xls
        if filename.endswith(".xls"):
            # Pesquisar o padrão
            match = PATTERN.search(filename)
            # Se verdadeiro, capturar o dígito da string e utilizá-lo para obter o valor da chave do dict `meses`
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                # Novo nome do arquivo após substituição
                new_name = PATTERN.sub(f"{name_month}", filename).replace(".xls", ".csv")

                # Caminhos completos dos arquivos antigo e novo
                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path

In [27]:
input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

file_path_generator = process_file(input_dir, output_dir)

# Passa as estruturas de dados pra read_csv realizar as operações na importação:
datasets = [pd.read_csv(file, sep='\t',
                        encoding="UTF-16 LE",
                        dtype=DTYPES,
                        parse_dates=DATE_TYPES,
                        date_format=DATE_FORMAT,
                        converters=CONVERTERS)
            for file in file_path_generator]

df_temp = pd.concat(datasets, axis=0, ignore_index=True)


In [28]:
# https://pandas.pydata.org/docs/user_guide/text.html#string-methods 
# Transforma os caracteres em maiúsculo
for c in TO_UPPER_LIST:
    df_temp[c] = df_temp[c].str.upper()
    df_temp.columns.str.strip()


pd.set_option("display.max_columns", None)

print(df_temp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ANO_BO                    119158 non-null  string        
 1   NUM_BO                    119158 non-null  string        
 2   NUMERO_BOLETIM            119158 non-null  string        
 3   BO_INICIADO               119158 non-null  datetime64[ns]
 4   BO_EMITIDO                119158 non-null  datetime64[ns]
 5   DATAOCORRENCIA            119158 non-null  datetime64[ns]
 6   HORAOCORRENCIA            110412 non-null  object        
 7   PERIDOOCORRENCIA          119158 non-null  string        
 8   DATACOMUNICACAO           119158 non-null  datetime64[ns]
 9   DATAELABORACAO            119158 non-null  datetime64[ns]
 10  BO_AUTORIA                119158 non-null  string        
 11  FLAGRANTE                 119158 non-null  string        
 12  NU

In [9]:
df_temp.sample(5)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
108546,2023,428064,428064/2023,2023-03-19 19:52:59,2023-03-19 19:53:04,2023-03-17,23:30,A NOITE,2023-03-18,2023-03-19 19:52:59,DESCONHECIDA,NÃO,<NA>,RUA RODRIGUES DE MEDEIROS,200,PEDREIRA,S.PAULO,SP,NaN,NaN,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,98º D.P. JARDIM MIRIAM,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE
80291,2023,1641,1641/2023,2023-04-12 00:37:11,2023-04-12 00:37:11,2023-04-11,19:30,A NOITE,2023-04-11,2023-04-12 00:37:11,CONHECIDA,SIM,1529/2023 - 10330,RUA LAPLACE,320,CAMPO BELO,S.PAULO,SP,-23.630996,-46.677351,VIA PÚBLICA,NaN,BO PARA FLAGRANTE,"27º DP ""DR. IGNÁCIO FRANCISCO""","27º DP ""DR. IGNÁCIO FRANCISCO""",TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - VEICULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,SAMSUNG
77240,2023,1794,1794/2023,2023-04-06 18:48:58,2023-04-06 18:48:58,2023-03-15,14:30,A TARDE,2023-03-15,2023-04-06 18:48:58,CONHECIDA,NÃO,1379/2023 - 10362,AVENIDA PROFESSOR LUIZ IGNÁCIO ANHAIA ME,8900,JARDIM INDEPENDÊNCIA (SÃO PAULO),S.PAULO,SP,-23.583247,-46.552452,VIA PÚBLICA,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,69º D.P. TEOTONIO VILELA,70º D.P. VILA EMA,TÍTULO I - PESSOA (ARTS. 121 A 154),HOMICÍDIO SIMPLES (ART. 121),MORTE DECORRENTE DE INTERVENÇÃO POLICIAL (RES....,CONSUMADO,AUTOR/VITIMA,SIM,<NA>,<NA>,MASCULINO,2002-05-18,20,<NA>,<NA>,<NA>,NaN,HOMICÍDIO SIMPLES (ART. 121),AUTOR/VITIMA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,SAMSUNG
17207,2023,699747,699747/2023,2023-05-06 18:04:13,2023-05-06 18:04:22,2023-05-06,15:50,A TARDE,2023-05-06,2023-05-06 18:04:13,DESCONHECIDA,NÃO,<NA>,AVENIDA CASA GRANDE,583,CASA GRANDE,DIADEMA,SP,-23.706264,-46.594000,VIA PÚBLICA,NaN,BO PARA INVESTIGAÇÃO,DELEGACIA ELETRONICA,02º D.P. DIADEMA,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - VEICULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,FSV3E17,SP,DIADEMA,PRATA,HYUNDAI/CRETA 16A ACTION,2022,<NA>,AUTOMOVEL,1,MOTOROLA
48682,2023,568,568/2023,2023-06-20 16:59:42,2023-06-20 16:59:42,2023-06-19,22:00,A NOITE,2023-06-20,2023-06-20 16:59:42,DESCONHECIDA,NÃO,<NA>,RUA BAHIA,100,TARUMÃ,JUNDIAI,SP,-23.171517,-46.869381,VIA PÚBLICA,NaN,APRECIAÇÃO DO DELEGADO TITULAR,03º D.P. JUNDIAI,03º D.P. JUNDIAI,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - TRANSEUNTE,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,MOTOROLA


In [20]:
dt_nasc = df_temp[df_temp["DATANASCIMENTO"].notnull()]

In [22]:
dt_nasc["DATANASCIMENTO"]

48        1975-09-23
300       1966-07-04
301       1966-07-04
761       1991-12-22
1573      1999-03-13
             ...    
116495    2000-01-17
116496    2000-01-17
116808    1967-02-28
118280    1952-11-22
118281    1952-11-22
Name: DATANASCIMENTO, Length: 342, dtype: object

### Lidando com valores ausentes


Definir como lidar com dados ausentes é uma tarefa dependente dos objetivos da análise. A ausência de dados pode adiar a análise até que uma nova coleta seja feita, ou pior: pode resultar em uma análise enviesada que inviabiliza o trabalho. Por isso, compreender claramente os motivos dos dados ausentes é vital para garantir não apenas a imparcialidade da análise, mas a estratégia mais eficáz de lidar com a ausência de informação.

Podem existir vários motivos pelos quais determinados valores estão faltando no conjunto de dados, por exemplo:

- Os dados podem ser corrompidos devido a manutenção inadequada;
- As observações não são registradas ou não são fornecidas por algum motivo específico desde a proteção de informações sensíveis, até alguma falha humana;
- O usuário pode não fornecer a informação por acidente ou intensionalmente, se recusando a passar os dados;

Formalmente, existem três suposições que os estatísticos fazem para compreender os motivos dos dados ausentes: MCAR (Missing Completely At Random), MAR (Missing At Random) e MNAR (Missing Not At Randon).

#### MCAR ou Ausência completamente aleatória

- MCAR (Missing Completely At Random): Podem ser considerados MCAR os dados ausentes por uma razão completamente aleatória e imprevisível, sem nenhum motivo sistemático ou qualquer padrão discernível. Por exemplo, alguma falha no sistema de coleta ou equipamento de registro, perda da amostra ou algum detalhe técnico insatisfatório.   
Neste caso, esses valores podem ser descartados com segurança, pois não irão impactar na imparcialidade da análise.

#### MAR ou Ausência aleatória

- MAR (Missing At Random): Podem ser classificados como MAR, os dados ausentes que têm alguma relação com outras variáveis, ou seja, o motivo dos valores ausentes pode ser explicado analisando as informações obtidas para outras variáveis, pois existe alguma relação entres os valores ausentes e outros valores. Neste caso, os dados não faltam em todas as amostras, mas nas subamostras dos dados. Além disso, há algum padrão que permite a suposição e a imputação de dados com segurança. 

Suponha que estamos analisando os dados sobre roubos de celulares em várias delegacias de diferentes bairros de uma cidade. A variável "DELEGACIA_CIRCUNSCRICAO" representa o bairro em que o Boletim de Ocorrência foi registrado, e a variável "ANO_FABRICACAO" se refere ao ano de fabricação do celular roubado.

Agora, imagine que temos duas delegacias, "Delegacia A" e "Delegacia B". A "Delegacia A" está localizada em um bairro mais antigo, onde os roubos de celulares costumam envolver dispositivos mais antigos, enquanto a "Delegacia B" está localizada em um bairro mais moderno, onde os roubos geralmente envolvem dispositivos mais recentes.

Nesse cenário hipotético, poderíamos observar o seguinte:

- Na "Delegacia A", a maioria dos registros de roubos de celulares tem valores ausentes em "ANO_FABRICACAO". Isso ocorre porque os criminosos frequentemente roubam celulares mais antigos nessa área, e as vítimas podem não se lembrar do ano de fabricação ou acharem essa informação menos relevante.

- Na "Delegacia B", a maioria dos registros de roubos de celulares tem mais dados preenchidos em "ANO_FABRICACAO", porque os celulares roubados nessa área tendem a ser mais novos, e as vítimas podem estar mais familiarizadas com as características de seus dispositivos.

Nesse cenário, a ausência de dados em "ANO_FABRICACAO" não está diretamente relacionada ao valor específico de "DELEGACIA_CIRCUNSCRICAO" em si, mas sim à natureza dos roubos de celulares em diferentes bairros. Assim, pode-se argumentar que a ausência de dados em "ANO_FABRICACAO" é aleatória condicionalmente às categorias de "DELEGACIA_CIRCUNSCRICAO", tornando "ANO_FABRICACAO" MAR em relação a "DELEGACIA_CIRCUNSCRICAO".      

#### MNAER ou Ausência não aleatória

- MNAR (Missing Not At Randon): São considerados MNAR os dados que estão ausentes por se relacionarem com a própria variável observada. Ou seja, a probalilidade de estar ausente não é aleatória e tem a ver com a própria variável, diferentemente do MAR. Por exemplo uma vítima de roubo de celular do sexo feminino, pode ter uma maior probabilidade de fornecer dados sobre `ESTADOCIVIL` em comparação com as vítimas do sexo masculino. Nesse caso, a ausência dos dados sobre o estado civil está relacionada com a variável `SEXO`, tornando os dados sobre o estado civil MNAR. Se, por algum motivo, realizarmos uma análise estatística sobre o estado civil das vítimas de roubo de celular em SP, sem levar em conta o fato de que homens tendem a não informar o estado civíl, provavelmente teríamos uma conclusão enviezada. Poderíamos concluir, por exemplo, que os dados mostram uma onda de assaltos a mulheres casadas.  

No nosso caso, vamos apenas substituir os valores <NA>, <NaN> e <NAT> por "Não Informado". 

In [15]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ANO_BO                    119158 non-null  string        
 1   NUM_BO                    119158 non-null  string        
 2   NUMERO_BOLETIM            119158 non-null  string        
 3   BO_INICIADO               119158 non-null  datetime64[ns]
 4   BO_EMITIDO                119158 non-null  datetime64[ns]
 5   DATAOCORRENCIA            119158 non-null  datetime64[ns]
 6   HORAOCORRENCIA            110412 non-null  object        
 7   PERIDOOCORRENCIA          119158 non-null  string        
 8   DATACOMUNICACAO           119158 non-null  datetime64[ns]
 9   DATAELABORACAO            119158 non-null  datetime64[ns]
 10  BO_AUTORIA                119158 non-null  string        
 11  FLAGRANTE                 119158 non-null  string        
 12  NU

In [18]:
df_temp["NUMERO_BOLETIM_PRINCIPAL"].value_counts() 

NUMERO_BOLETIM_PRINCIPAL
1374/2023 - 70309      132
1292/2023 - 10357      108
1247/2023 - 10371      100
2647/2023 - 10335       96
3618/2023 - 10216       90
                      ... 
109/2023 - 100908        1
84424/2023 - 900021      1
65419/2023 - 900022      1
654/2023 - 30104         1
16/2023 - 80505          1
Name: count, Length: 14552, dtype: Int64

In [27]:
valid_boletin_principal = df_temp["NUMERO_BOLETIM_PRINCIPAL"].notna()

In [28]:
valid_boletin_principal[:5]

0    False
1    False
2    False
3    False
4    False
Name: NUMERO_BOLETIM_PRINCIPAL, dtype: bool

In [29]:
valid_boletin_principal.sum()

37625

In [12]:
count_null_values = df_temp.isnull().sum()

In [14]:
count_null_values.sort_values(ascending=False)[0:20]

CORCUTIS              119158
RELACIONAMENTO        119158
EXAME                 119158
PARENTESCO            119158
NATURALIDADE          119150
NACIONALIDADE         119131
GRAUINSTRUCAO         119097
PROFISSAO             119041
ESTADOCIVIL           119012
DESDOBRAMENTO         118957
IDADE                 118816
DATANASCIMENTO        118816
SEXO                  118781
TIPOVINCULO           118780
NATUREZAVINCULADA     118780
VITIMAFATAL           118780
TIPOPESSOA            118780
UF_VEICULO             92178
CIDADE_VEICULO         92178
DESCR_TIPO_VEICULO     92121
dtype: int64

---

## Referencias

- [Link da documentação para trabalhar a consistencia das datas](https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html)